In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Set random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

# Define paths to the image folders
healthy_folder = '/content/drive/My Drive/ALL/Endoscopic/antralulcer/Healthy Antral picture'
unhealthy_folder = '/content/drive/My Drive/ALL/Endoscopic/antralulcer/Antral Ulcer(Unhealthy)'

# Load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        images.append(img_array)
    return np.array(images)

healthy_images = [os.path.join(healthy_folder, img) for img in os.listdir(healthy_folder)]
unhealthy_images = [os.path.join(unhealthy_folder, img) for img in os.listdir(unhealthy_folder)]

# Combine images and labels
all_images = healthy_images + unhealthy_images
all_labels = [0] * len(healthy_images) + [1] * len(unhealthy_images)

# Convert image paths to feature vectors
X = load_and_preprocess_images(all_images)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X.reshape(-1, np.prod(X.shape[1:])), all_labels)

# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=1)

# Convert labels to numpy arrays
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

# Define MobileNetV2 base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom head for binary classification
model = Sequential([ base_model, GlobalAveragePooling2D(), Dense(512, activation='relu'), Dense(1, activation='sigmoid')])

# Freeze base model layers
base_model.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Reshape the feature vectors to match the input shape expected by the MobileNetV2 model
train_images = train_images.reshape(-1, 224, 224, 3)
val_images = val_images.reshape(-1, 224, 224, 3)

# Train the model
history = model.fit(
    train_images, train_labels,
    epochs=12,
    validation_data=(val_images, val_labels)
)

# Save the trained model
model.save('/content/drive/My Drive/ALL/Endoscopic/antralulcer/model_mobilenet_smote.h5')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.5291 - loss: 1.7126 - val_accuracy: 0.7143 - val_loss: 0.4782
Epoch 2/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.7739 - loss: 0.4561 - val_accuracy: 0.8571 - val_loss: 0.2761
Epoch 3/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8419 - loss: 0.3710 - val_accuracy: 0.8980 - val_loss: 0.2928
Epoch 4/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8953 - loss: 0.2485 - val_accuracy: 0.9796 - val_loss: 0.1767
Epoch 5/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9204 - loss: 0.1973 - val_accuracy: 0.9184 - val_loss: 0.1977
Epoch 6/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9543 - loss: 0.1310 - val_accuracy: 0.9184 - val_loss: 0.1495
Epoch 7/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9775 - loss: 0.0994 - val_accuracy: 0.9388 - val_loss: 0.1650
Epoch 8/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9973 - loss: 0.0712 -

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Add this import if not already present

# Define a function to preprocess the input image and predict its class
def predict_image(image_path):
    model = tf.keras.models.load_model('/content/drive/My Drive/ALL/Endoscopic/antralulcer/model_mobilenet_smote.h5')
    img = load_img(image_path, target_size=(224, 224))  # Use load_img directly
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Ensure you preprocess as during training
    prediction = model.predict(img_array)
    if prediction > 0.5:
        return "Unhealthy Antral"
    else:
        return "Healthy Antral"

# Example usage:
image_path = '/content/9.JPG'  # Replace with the path to your image
prediction = predict_image(image_path)
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: Unhealthy Antral
